# import new/updated characteristics from google sheet file exported as csv


In [7]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

In [97]:
import csv
csv_file_path='./20200608 SEARCH-by strat. representation-Bin. obs_v1 - MiCorr.csv'
with open(csv_file_path,"r") as f:
    reader =csv.DictReader(f)
    print(reader.fieldnames)

def to_uid(name):
    camel_case_name=''.join(name.title().split(' '))

    return camel_case_name[0].lower() + camel_case_name[1:]


['Characteristic', 'Sub-characteristic', 'Observation mode', 'Values', 'Sub-values', 'Image', 'Variables', 'unit', 'uid', 'Definition from observation under binocular', 'Definition from observation on cross-section', 'Notice for modification of the current MiCorr interface', 'Comments', 'CP Binocular', 'CP CS', 'CM Binocular', 'CM CS', 'D Binocular', 'D CS', 'M Binocular', 'M CS', 'NMM Binocular', 'NMM CS', 'POM Binocular', 'POM CS', 'S Binocular', 'S CS', 'SV Binocular', 'SV CS']


# import image urls from html export


In [45]:
name_img_url = {}

In [ ]:
from bs4 import BeautifulSoup
f=open('20200608 SEARCH-by strat. representation-Bin. obs_v1/MiCorr.html')
content=f.read()
soup=BeautifulSoup(content,'html.parser')
imgs=soup.find_all('img')
name_img_url={}
for im in imgs:
    parent_td=im.parent.parent if im.parent.parent.name=='td' else None
    if parent_td:
        name_img_url[parent_td.previous_sibling.previous_sibling.text]=im['src']

In [90]:

def new_characteristic(family_uid, uid, name, order, description=None, image_url=None, optgroup=None):
    cql_update_row = f"""{{family_uid:'{family_uid}', uid:'{uid}', name:'{name}', order:{order}"""
    arg_list=[cql_update_row]

    if image_url is None:
        # search characteristic name in name to image_url map
        image_url = name_img_url.get(name)
    if description:
        arg_list.append(f"description:'{description}'")
    if image_url:
        arg_list.append(f"image_url:'{image_url}'")
    if optgroup:
        arg_list.append(f"optgroup:'{optgroup}'")

    return ', '.join(arg_list) + '},'

def new_family(fg_uid, uid, name, order, observation, description='',visible=False, variable=False, unit=None, fieldset=None):
    # adapt csv column value to cql format
    observation=observation.lower()
    if observation=='cross-section':
        observation='cross_section'
    cql_update_row = f"""{{fg:'{fg_uid}', order:{order}, observation:{observation}, uid:'{uid}', name:'{name}', visible: {visible}, variable: {variable}, description:'{description}'"""
    arg_list=[cql_update_row]
    if unit:
        arg_list.append(f"unit:'{unit}'")
    if fieldset:
        arg_list.append(f"fieldset:'{fieldset}'")
    return ', '.join(arg_list) + '},'


## Microstructure Binocular special case

In [ ]:

# we treat all values and sub-values as characteristics (with optgroup + property for Cristalline microstructure)
# and optgroup - (closure of previous group) for isolated aggregate


with open(csv_file_path,"r") as f:
    reader =csv.DictReader(f)
    current_value=''
    characteristic_order = 1
    family_order = 1
    sub_value_order = 1
    optgroup_level = 0
    description_field_in = {'Binocular':'Definition from observation under binocular',
                           'Cross-section': 'Definition from observation on cross-section'}

    for r in reader:
        if 'Microstructure' not in r['Characteristic']:
            continue
        observation = r['Observation mode']
        if observation != 'Binocular':
            continue

        if r['Values'] == '' and r['Sub-values'] == '':
            family_uid='microstructureFamily'
            cql_family_update_row=new_family(fg_uid='fgMicrostructure', uid=family_uid, name='Microstructure',
                                     order=family_order,observation='binocular', description=r[description_field_in[observation]])
            print(cql_family_update_row)
            print(f'\n// {family_uid} characteristics:')
            family_order+=1

        if r['Values'] != '':
            # print('\nNew characteristic:\n')
            current_value = r['Values']
            current_value_uid = to_uid(r['Characteristic'] + ' ' +current_value)+'Characteristic'
            # trick to detect end of optgroup in our particular case (1 optgroup only yet)
            if optgroup_level:
                optgroup=None
                optgroup_level-=1

            # print(new_characteristic(family_uid=family_uid, uid=current_value_uid,
            #                         name=current_value,order=characteristic_order,
            #                         description=r[description_field_in[observation]],optgroup=optgroup))
            # family_order+=1
            # characteristic_order = 1
            characteristic_order += 1

        # print(observation, r['Characteristic'], current_value,'None' if r['Sub-values'] == '' else r['Sub-values'])
        current_subvalue = r['Sub-values']
        if observation=='Binocular' and current_value!='' and current_subvalue!='':
            if sub_value_order==1:
                optgroup_level+=1
                optgroup=current_value
                print(f'// {current_value_uid} optgroup options:')
            current_subvalue_uid = to_uid(current_value + ' ' +current_subvalue)+'Characteristic'
            print(new_characteristic(family_uid=family_uid, uid=current_subvalue_uid,name=current_subvalue,
                                     order=characteristic_order))
            sub_value_order+=1
            characteristic_order+=1

In [92]:

FAMILY = 1
CHARACTERISTIC = 2

def import_family_group(family_group, filter_observation=None, fieldsets=[], filter_print=FAMILY|CHARACTERISTIC):
    with open(csv_file_path,"r") as f:
        reader =csv.DictReader(f)
        current_value=''
        characteristic_order = 1
        family_order = 1
        sub_value_order = 1
        optgroup = None
        optgroup_level = 0
        description_field_in = {'Binocular':'Definition from observation under binocular',
                               'Cross-section': 'Definition from observation on cross-section'}
        fieldset = None # group Values below under fieldset

        for r in reader:
            if family_group not in r['Characteristic']:
                continue
            observation = r['Observation mode']
            if observation != filter_observation:
                continue
            # print(f'family_group={r["Characteristic"]}')
            # print(f'observation={observation}')
            if r['Values'] == '' and r['Sub-values'] == '' and r['Sub-characteristic']!='':
                family_name=r['Sub-characteristic']
                if family_name in fieldsets:
                    fieldset = family_name
                    print(f'// new fieldset={fieldset}')
                else:
                    print(f'// new family_name={family_name}')
                    fieldset = None
                    family_uid=to_uid(r['Characteristic'] + ' ' +family_name)+'CSFamily'
                    cql_family_update_row=new_family(fg_uid='fg'+family_group, uid=family_uid, name=family_name,
                                         order=family_order,observation=observation, description=r[description_field_in[observation]])
                    if filter_print & FAMILY:
                        print(cql_family_update_row)
                    ## print(f'\n// {family_uid} characteristics:')
                family_order+=1
            if r['Values'] != '':
                current_value = r['Values']
                if r['Variables'] !='':
                    #print(f'\nNew {current_value} variable Family:\n')
                    current_value_uid = to_uid(r['Characteristic'] + ' ' +current_value)+'CSVarFamily'
                    cql_family_update_row=new_family(fg_uid='fg'+family_group, uid=current_value_uid, name=current_value,
                        order=family_order,observation=observation.lower(),
                        description=r[description_field_in[observation]], variable=True, unit=r['unit'], fieldset=fieldset)
                    if filter_print & FAMILY:
                        print(cql_family_update_row)
                    family_order+=1
                else:
                    # print(f'\nNew {current_value} characteristic:\n')
                    current_value_uid = to_uid(r['Characteristic'] + ' ' +current_value)+'CSCharacteristic'
                
                    # trick to detect end of optgroup in our particular case (1 optgroup only yet)
                    if optgroup_level:
                        optgroup= None
                        optgroup_level-=1

                    cql_characteristic_update_row = new_characteristic(family_uid=family_uid, uid=current_value_uid,
                                         name=current_value,order=characteristic_order,
                                         description=r[description_field_in[observation]],optgroup=optgroup)
                    if filter_print & CHARACTERISITIC:
                        print(cql_characteristic_update_row)
                    
                    # family_order+=1
                    # characteristic_order = 1
                    characteristic_order += 1

            

## Interface cross-section special case

In [96]:
# we treat Roughness as a fieldset (as it includes multiple families instead of characteristics)
# to do add group/fieldset column to spreadsheet

import_family_group('Interface', filter_observation='Cross-section', fieldsets=['Roughness'], filter_print=CHARACTERISTIC) #FAMILY|CHARACTERISTIC

// new family_name=Profile
{family_uid:'interfaceProfileCSFamily', uid:'interfaceBlankCSCharacteristic', name:'Blank', order:1},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceStraightCSCharacteristic', name:'Straight', order:2, description:'No waves along the observed length.'},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceWavyCSCharacteristic', name:'Wavy', order:3, description:'Shows variations or undulations with a low height to length ratio. Angles are greater than 90°.'},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceBumpyCSCharacteristic', name:'Bumpy', order:4, description:'Presents variations that are close in height and length. Angles can be equal to or less than 90°.'},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceIrregularCSCharacteristic', name:'Irregular', order:5, description:'Presents variations whose height and length are unrelated. Angles may be variable.'},
// new family_name=Transition
{family_uid:'interfaceTransitionCSFamily', uid:'